# Choose settings

##### Choose your settings here

In [1]:
datasets_path = '/content/drive/My Drive/FM/datasets/'
MERGE_PICKLES = ['buffer_ALIGN_traffic-signs_upto-8192.pkl', 'buffer_ALIGN_traffic-signs_upto-12288.pkl', 'buffer_ALIGN_traffic-signs_upto-13417.pkl']
OUTPUT_FILE = 'ALIGN_traffic-signs.pkl'

# Load libraries

In [2]:
import torch
import numpy as np
from matplotlib import pyplot as plt
import os
from google.colab import drive
import pickle

print("Torch version:", torch.__version__)

Torch version: 2.1.0+cu121


# Mounting storage

In [3]:
drive.mount('/content/drive')
!ls "{datasets_path}"

Mounted at /content/drive
AlexNet_cats-vs-dogs-large.pkl		   image_embeddings__traffic-signs.pkl
AlexNet_imagenet-subset.pkl		   imagenet_one-of-each-class-except-cats-and-dogs
AlexNet_traffic-signs.pkl		   indizes_clean_Anni.txt
ALIGN_cats-vs-dogs-large.pkl		   jellyfish-classification
ALIGN_imagenet-subset.pkl		   mislabeled_instances_cats-vs-dogs.pkl
buffer_ALIGN_traffic-signs_upto-12288.pkl  note.txt
buffer_ALIGN_traffic-signs_upto-13417.pkl  text_dog_embeddings.pkl
buffer_ALIGN_traffic-signs_upto-8192.pkl   text_random_embeddings.pkl
cats-dogs-big_ids.pkl			   traffic-signs
cats-dogs-big.pkl			   train-small
cats-vs-dogs-large			   ViT-CLS_cats-vs-dogs-large.pkl
CLIP_cats-vs-dogs-large.pkl		   ViT-CLS_cats-vs-dogs-large-SUBSET.pkl
CLIP_imagenet-subset.pkl		   ViT-CLS_imagenet-subset.pkl
CLIP_traffic-signs.pkl			   ViT-CLS_traffic-signs.pkl
dog_wrong_2_12.txt			   ViT-pooling_cats-vs-dogs-large.pkl
dog_wrong.txt				   ViT-pooling_imagenet-subset.pkl
image_embeddings__cats-vs-dogs.p

In [4]:
merged_embeddings = {}

for file in MERGE_PICKLES:
  with open(datasets_path + file, 'rb') as f:
    print(f'Processing file {file}')
    image_embeddings = pickle.load(f)
    for key in image_embeddings:
      if key not in merged_embeddings:
        merged_embeddings[key] = []
      merged_embeddings[key].append(image_embeddings[key])
    del image_embeddings
    torch.cuda.empty_cache()

Processing file buffer_ALIGN_traffic-signs_upto-8192.pkl
Processing file buffer_ALIGN_traffic-signs_upto-12288.pkl
Processing file buffer_ALIGN_traffic-signs_upto-13417.pkl


In [5]:
for key in merged_embeddings:
  tensors = []
  for list_of_tensors in merged_embeddings[key]:
    for tensor in list_of_tensors:
      tensors.append(tensor.cpu().type(torch.float))
  merged_embeddings[key] = torch.cat(tensors)
  print(key, merged_embeddings[key].shape)

30 kilometers per hour speed limit traffic sign torch.Size([2220, 640])
80 kilometers per hour speed limit traffic sign torch.Size([1860, 640])
100 kilometers per hour speed limit traffic sign torch.Size([1440, 640])
give way traffic sign torch.Size([2160, 640])
no entry traffic sign torch.Size([1109, 640])
no overtaking traffic sign torch.Size([1470, 640])
priority over oncoming traffic sign torch.Size([2099, 640])
stop sign torch.Size([1050, 640])


In [6]:
with open(datasets_path + OUTPUT_FILE, 'wb') as f:
  pickle.dump(merged_embeddings, f)